<h1><center>DS UA 9201 Problem Set 2</center></h1>
<h1><center>Grace Yang (gy654)</center></h1>
<h1><center>Due: Nov 2nd</center></h1>

# Exercise1: Directed Acyclic Graph

## Question1 Colliders:
Of the five variables in the graph, 2 are colliders with respect to a path and 3 are non colliders. Which variables are colliders with respect to a path and which are non colliders?

- $M$ and $Y$ are two colliders with respect to the path $T \to W$. 
- $X$ is a non-collider.

## Question2 Identification:
Suppose that we wanted to estimate the effect of $T$ on $Y$. Indicate if we should or should not condition on X, and explain why, and indicate if we should or should not condition on M and explain why. 

We should condition on $X$, since the backdoor path $T\leftarrow X \rightarrow Y$ should be blocked so that we could estimate the effect of $T$ on $Y$. Conditioning on $X$ can block that path and make $T$ and $Y$ conditionally independent given $X$.

However, we should not condition on $M$ since $M$ is a post-treatment variable. Conditioning on $M$ would induce post-treatment selection bias. 

## Question3: Path
Suppose that we wanted to estimate the effect of $M$ on $Y$. List all the backdoor paths between $M$ and $Y$, and indicate which variable we should condition on to block each path. There may be multiple valid options for each path.

All backdoor path:

- $M \leftarrow W \rightarrow Y$ Condition on $W$
- $M \leftarrow W \leftarrow T \leftarrow X \rightarrow Y$ Condtion on $W$ or $T$ or $X$
- $M \leftarrow  T  \rightarrow W \rightarrow Y$ Condition on $T$ or $W$
- $M \leftarrow T \leftarrow X \rightarrow Y$ Contion on $T$ or $X$

# Exercier2: Social Pressure and Voter Turnout

Consider again the GOTV data from last problem set by Gerber, Green and Larimer. Although it is not specified in the paper, it is highly possible that the authors created subgroups based on the turnout history for 5 previous primary and general elections (number of times the individual voted), and number of registered voters in the household. In this problem, we will create subgroups based on the turnout history, and investigate the CATE(conditional average treatment effect) and the effect modifications in each subgroup. We denote the turnout history/number of times voted as a covariate $X_i$ for individual $i$

## Question1: Data preparation

Construct a new dataset for this problem using individual dataset from the last problem set. 

1. Create a new column num_voted to represent the number of times the individual has voted in previous 5 elections by summing the variables g2000, p2000, g2002, p2002, and p2004(excluding g2004), the resulting column should be an integer ranging from $[0, 5]$

In [1]:
import pandas as pd
file_path = './gotv_individual.csv'
gotv = pd.read_csv(file_path)


In [2]:
gotv['num_voted']= gotv[['g2000', 'p2000','g2002', 'p2002', 'p2004']].sum(axis=1)

2. In the following problems, we are using the individual data with num_voted as different subgroups. To simplify the probelem, we investigate only the "Neighbor" treatment effect. Construct a cleaner dataset with id, hh_id, hh_size, num_voted, voted, treatment as columns and fulter out treatment groups besides {Neighbor, Control}

In [3]:
gotv = gotv[['hh_id', 'hh_size', 'num_voted', 'voted', 'treatment']]
gotv = gotv[(gotv['treatment'] == ' Neighbors')| (gotv['treatment'] == ' Control')]

3. Construct a household-level dataset by taking the means of hh_size, num_voted, and voted in each household(the other variables are all equal within the same household and can simply be left as they are). Round the mean of num_voted up to the nearest integer. Your resulting dataset should have one household per row, and hh_id, hh_size, num_voted, voted, and treatment as columns. The variable num_voted should have only values 0,1,2,3,4,5


In [4]:
household_df = gotv.groupby(['hh_id', 'treatment']).mean()
household_df['num_voted'] = household_df['num_voted'].map(lambda x: round(x))
household_df

,,hh_size,num_voted,voted
hh_id,treatment,,,
3.0,Control,3.0,2,0.666667
4.0,Control,2.0,2,0.000000
5.0,Control,1.0,3,1.000000
7.0,Control,1.0,2,0.000000
8.0,Control,2.0,4,1.000000
...,...,...,...,...
179992.0,Control,2.0,4,0.000000
179997.0,Neighbors,2.0,3,0.000000
179998.0,Control,2.0,3,1.000000


4. Report number of households in each subgroup for both treatment and control, what do you observe?

In [5]:
n_c = household_df.index.get_level_values(1).value_counts()[' Control']
n_n = household_df.index.get_level_values(1).value_counts()[' Neighbors']
print(f'Number of households in control group: {n_c}')
print(f'Number of households in treatment group: {n_n}')


Number of households in control group: 99999
Number of households in treatment group: 20000


Observe that the dataset is imbalanced in terms of treatment distribution. The dataset contains far more households within the control group than households in the treatment group.

## Question2 CATE for subgroups
We defind conditional average treatment effect as the ATE for different subgroups defined by the "num_voted" variable:

$$\tau(x) = E[Y_i(1) - Y_i(0)|X_i = x], x \in \{0,1,2,3,4,5\}$$
Since treatment was randomized at the household level, positivity and ignorability hold both unconditionally, and conditionally within each subgroup. For each subgroup: 
1. Estimate the CATE and report the variance of your estimates.

In [6]:
household_df.reset_index(level=1,  inplace=True)

In [24]:
import numpy as np
def cate(num_vote):
    subgroup = household_df[household_df['num_voted']==num_vote]
    control = subgroup[subgroup['treatment']== ' Control']
    neighbor = subgroup[subgroup['treatment']== ' Neighbors']
    cate = neighbor.voted.mean() - control.voted.mean()
    std = np.sqrt((np.var(control.voted, ddof = 1)/ len(control)) + (np.var(neighbor.voted, ddof = 1)/ len(neighbor)))
    n_x = len(neighbor)
    return cate, std, n_x

x_dict = {}
for i in range(6):
    cate_x, std, n_x = cate(i)
    x_dict[i] = (cate_x,  std, n_x)
    print(f'For subgroup with num_voted = {i}: the CATE is: {cate_x}, variance is:{std**2}')



For subgroup with num_voted = 0: the CATE is: 0.10153297859448504, variance is:0.0025216276632681906
For subgroup with num_voted = 1: the CATE is: 0.06864983709083408, variance is:0.00017474136062828913
For subgroup with num_voted = 2: the CATE is: 0.06859163081337721, variance is:2.4760661601173454e-05
For subgroup with num_voted = 3: the CATE is: 0.09737209120206786, variance is:3.857050905862662e-05
For subgroup with num_voted = 4: the CATE is: 0.09871522216345402, variance is:5.236684122055182e-05
For subgroup with num_voted = 5: the CATE is: 0.04037397996673986, variance is:0.0012530984345875819


2. Construct an asympotic 95% confidence interval around your estimates.

In [8]:
from scipy.stats import norm
scalar = norm.ppf(0.975)
for k, v in x_dict.items():
    print(f'Confidence interval of CATE for subgroup with num_voted = {k}: {[v[0]- scalar * v[1], v[0]+ scalar * v[1]]}')


Confidence interval of CATE for subgroup with num_voted = 0: [0.003111797797016924, 0.19995415939195316]
Confidence interval of CATE for subgroup with num_voted = 1: [0.04274111768740477, 0.0945585564942634]
Confidence interval of CATE for subgroup with num_voted = 2: [0.05883883316787825, 0.07834442845887618]
Confidence interval of CATE for subgroup with num_voted = 3: [0.08519970350523845, 0.10954447889889726]
Confidence interval of CATE for subgroup with num_voted = 4: [0.08453195511673388, 0.11289848921017416]
Confidence interval of CATE for subgroup with num_voted = 5: [-0.029007040743137757, 0.10975500067661748]


3. What conclusions can you draw from these statistics?

The treatment of neighbors has a statistically significant impact on subgroups with num_voted = 0,1,2,3,4, but not on subgroup with num_voted = 5 sicnce 0 is contained in its confidence interval. It makes sense that the treatment's impacts on extreme subgroups( num_voted = 0 or 5, i.e. people who always vote and people who never vote) are less statistically significant because these subgroups's voting belief are harder to reverse.

## Question3: Effect modification
Suppose we want to estimate whether there is a difference in effects for two extreme groups, ndividuals who always vote $(X_i = 5)$ and individuals who never vote $(X_i)$, we construct an estimator $\hat{\Delta}$ to estimate the difference. As we saw in class, we can estimate this difference as: 
$$\hat{\Delta} = \hat{\tau}(0) - \hat{\tau}(5)$$

1. Calculate the variance of $\hat{\Delta}$ and construct a 95% confidence interval around it, can we say that there's significant difference in the treatment effect for people who always vote and people who never vote?

$$\widehat{Var}[\widehat{\Delta}] = \widehat{Var}[\widehat{\tau}(x)] + \widehat{Var}[\widehat{\tau}(w)] $$

In [25]:
var_delta = x_dict[0][1]**2 + x_dict[5][1]**2
delta = x_dict[0][0] - x_dict[5][0]
interval = [delta - scalar * np.sqrt(var_delta), delta + scalar * np.sqrt(var_delta)]
print(f'delta hat is :{delta}')
print(f'The variance of delta estimator: {var_delta}')
print(f'The 95 % confidence interval is: {interval}')

delta hat is :0.06115899862774518
The variance of delta estimator: 0.0037747260978557725
The 95 % confidence interval is: [-0.05925883586662019, 0.18157683312211054]


We cannot say that there is significant difference in the treatment effect since the 95% confidenceinterval still contains 0.

2. Combine your observations with conclusions from Question2, comment about your findings. 

Results indicate that there isn't convincing evidence that the neighbors treatment affected extreme subgroups differently( people who always vote and people never vote), and for these two extreme subgroups, there isn't convincing evidence that the neighbor treatment effect is statistically significant. 

## Question4: Sample sizes and significance effect
In the experiment, the authors claimed no significant differences between groups, one possible reason may be that the sample size for each subgroup is too small. This is a practical problem we may encounter in experimental designs when we are testing multiple hypothesis or we are having too many subgroups. Explain in you own words why having more hypothesis/subgroups would make significant effect harder to detect for each group, assuming the overall sample size is fixed. 

When we are doing multiple hypothesis tests on too many subgroups, it is likely that the sample size for each subgroup is too small. We are likely to overestimate the CATE variance since sample size n is small in each strata. Consequently we overestimate the variance of the difference in treatment estimator, which is the sum of variance of CATE for each subgroup. The overestimation of estimator variance may lead to a lower test statistics and p-value, making it harder to detect the significant effec for each group. 

# Exercise 3: Social Pressure and Voter Turnout (continued):
## Question1:
Compute the ATE of the "Neighbors" treatment using the standard difference in mean estimator, i.e., $\hat{\tau} = \bar{T_t} - \bar{Y_c}$. Provide standard errors and 95% confidence intervals for your estimates.

In [20]:
control =household_df[household_df.treatment == ' Control']
neighbor =household_df[household_df.treatment == ' Neighbors']

ate = neighbor.voted.mean()- control.voted.mean()
std = np.sqrt(np.var(control.voted, ddof=1)/len(control) + np.var(neighbor.voted, ddof=1)/len(neighbor))
dim_var = std**2
interval = [ate - scalar * std, ate + scalar * std]
print(f'ATE:{ate}\n\
Standard Error:{std}\n\
95% confidence Interval:{interval}')


ATE:0.08478807597718846
Standard Error:0.0034012283714832453
95% confidence Interval:[0.07812179086588548, 0.09145436108849143]


## Question2:
Now compute the same ATE but with the stratification estimator that is defined as the weighted mean of the stratum CATEs that you computed in the previous problem.
$$\hat{\tau}_{block} = \sum_{x=0}^{5} \hat{\tau}(x)\frac{N_x}{N}$$
Compute variance and 95% confidence intervals for this estimator as well using the stratified variance estimator defined as:
$$\hat{Var}(\tau_{block}) = \sum_{x=0}^{5} \hat{Var}(\tau(x))(\frac{N_x}{N})^2$$
Comment on the difference between the ATE estimates you obtained here and in part a and their variances. What is it due to?

In [11]:
N = len(neighbor)
tau_block = 0
var_tau = 0
for v in x_dict.values():
    tau_block += v[0] * v[2]/N
    var_tau += v[1]**2 * (v[2]/N)**2
interval = [tau_block - scalar * np.sqrt(var_tau), tau_block + scalar * np.sqrt(var_tau)]
print(f'tau_block is :{tau_block}')
print(f'variance is :{var_tau}')
print(f'The 95% confidence interval: {interval}')

tau_block is :0.08491162608657876
variance is :1.1099029372716263e-05
The 95% confidence interval: [0.07838196573668582, 0.0914412864364717]


The value of the two estimators are similar, but the variance of the stratified ATE estimator($\hat{\tau}_{block}$) we obtained here is smaller than the variance of the completely randomized ATE estimator we obtained in part a. It is lower because if the stratification is correlated with the outcome(a household's previous voting history is correlated with a household's voting decision this year), then $S_{t, g}^2$ and $S_{c, g}^2$ will tend to be lower than their full-sample counterparts.

## Question3:
Now divide the dataset into 6 strata in such a way that each of the strata have same proportion of Treated and Control observations. You can do so by creating a new variable called "group" with values 0,1,2,3,4,5 and randomly assigning each value to $\frac{N_t}{6}$ treated units and $\frac{N_c}{6}$ control units. You may exclude enough treated and control units from the data to make $N_t$ and $N_c$ divisible by 6.
Compute the ATE by applying the estimator $\hat{\tau}_{block}$ to these newly created strata. Provide variance estmates block and 95% confidence intervals for these ATE estimates as well using the stratified variance estimator. Is the variance of this estimator much different from that of $\hat{\tau}$ you computed in Question1? Why do you think this is the case?

In [12]:
neighbor = neighbor.iloc[:19998, :]
control = control.iloc[:99996, :]
neighbor = neighbor.assign(group= np.random.permutation([0,1,2,3,4,5]*3333))
control = control.assign(group= np.random.permutation([0,1,2,3,4,5]* 16666))
combined = pd.concat([neighbor, control])
# combined.group.value_counts()

In [23]:

def random_tau_block(group): 
    subgroup = combined[combined.group == group]
    control = subgroup[subgroup.treatment == ' Control']
    neighbor = subgroup[subgroup.treatment == ' Neighbors']
    ate = neighbor.voted.mean()-control.voted.mean()
    std = np.sqrt(np.var(control.voted, ddof =1)/len(control) + np.var(neighbor.voted, ddof =1)/len(neighbor))
    interval = [ate - scalar * std ,ate + scalar * std ]
    return ate, std, interval

random_tau_block_dict = {}
for i in range(6):
    ate, std, interval = random_tau_block(i)
    random_tau_block_dict[i] = (ate, std, interval)
estimator_var = (1/6) * np.mean([k[1]**2 for k in random_tau_block_dict.values()])

print(f'variance_tau for randomized subgroups: {estimator_var}')
print(f'variance_tau in previous case: {var_tau}')
print(f'variance_tau from question 1: {dim_var}')

variance_tau for randomized subgroups: 1.1567512545864257e-05
variance_tau in previous case: 1.1099029372716263e-05
variance_tau from question 1: 1.1568354434982569e-05


The variance of tau for randomly divided subgroups is higher than the variance of tau in the previous case when subgroups are divided based on household's voting history. The variance is lower in the previous case because the voting history partially explains the difference in treatment effectes.

The variance in this case is similar as the variance of the difference in mean estimator in question1 since randomized stratification is like not doing stratification.

# Exercise 4: Estimand, estimator, bias, variance
## Question1 Estimators for the population mean:

Let us consider identically distributed variables $\{X_1, X_2, ..., X_n\}$ of mean $\mu$ and variance $\sigma^{2}$
Let us consider two estimators:
- the sample average $\bar{X} = \frac{1}{n}\sum_{i=1}^{n}X_i$
- the first observation (we disregard all the other observations) $x_1 = X_1$

1. A nice property of an estimator is unbiasedness. Show that both estimators are unbiased estimators of $\mu$(i.e their expectation is equal to $\mu$)

- $$E[\bar{X}] = E[\frac{1}{n}\sum_{i=1}^{n}X_i] = \frac{1}{n} \cdot n \cdot E[X_i] = \mu$$ 
since $X_i$ iids
- $$E[X_1] = \mu$$

Therefore both estimators are unbiased.

2. However, unbiasedness is not sufficient, and a criterion to distinguish a good estimator from a bad is its variance. Using basic properties of variance(variance of a sum of independent variables), establish the variance of each estimator in function of $\sigma$

$$Var(\bar{X}) = Var(\frac{1}{n}\sum_{i=1}^{n}X_i) = \frac{1}{n^2} Var(\sum_{i=1}^{n}X_i) = \frac{1}{n^2} \sum_{i=1}^{n}Var(X_i) = \frac{1}{n^2} \cdot n\sigma^2 = \frac{\sigma^2}{n} $$
$$Var(X_1) = \sigma^2$$

3. Which estimator should we use in pratice?

We should use the first estimator $\bar{X}$. The variance of the estimator shrink as sample size $n$ is sufficiently large. The asympototically normal behavior of the sample mean estimator ensures that we have more confidence in its estimation of the population mean as sample size increases. 

## Question 2: Estimators for causal effect:
With the simple example of estimators for the mean, we have illustrated that we can derive statistical properties of estimators to help us select the best one. In the previous homework you have shown that the IPW estimator is unbiased. In this homework, you will be guided through deriving the variance of this estimator, again usng simple properties of the variance. 
We are going to derive the variance of the oracle IPW estimator, in which the propensity scores are known(as if we had an oracle to tell us). and not estimated. Indeed, propensity score estimations breaks the independance between observations as we use all samples to estimate them(through logistic regression), and this makes the computation very complicated. 
We denote the oracle IPW estimator as:
$$\hat{\tau}^*_{IPW} = \frac{1}{n}\sum_{i=1}^{n}(\frac{T_iY_i}{e(X_i)}- \frac{(1-T_i)Y_i}{1-e(X_i)})$$
We are going to show that:
$$nVar[\hat{\tau}^*_{IPW}] = E[\frac{Y(1)^2}{e(X_i)}] + E[\frac{Y(0)^2}{1-e(X_i)}] - \tau^2$$

We decompose the proof as following:
1. Show that $$Var[\hat{\tau}^*_{IPW}] = \frac{1}{n}(Var[\frac{TY(1)}{e(X_i)}] + Var[\frac{(1-T)Y(0)}{1-e(X_i)}] - 2Cov[ \frac{TY(1)}{e(X_i)}, \frac{(1-T)Y(0)}{1-e(X_i)}])$$




$$Var[\hat{\tau}^*_{IPW}] = \frac{1}{n^2}\sum_{i=1}^{n}Var(\frac{T_iY_i}{e(X_i)} - \frac{(1-T_i)Y_i}{1-e(X_i)}) = \frac{1}{n} (Var[\frac{T_iY_i}{e(X_i)}] + Var[\frac{(1-T_i)Y_i}{1-e(X_i)}] - 2Cov(\frac{TY(1)}{e(X_i)}, \frac{(1-T)Y(1)}{1-e(X_i)}))$$
$$= \frac{1}{n}(Var[\frac{TY(1)}{e(X_i)}] + Var[\frac{(1-T)Y(0)}{1-e(X_i)}] - 2Cov[ \frac{TY(1)}{e(X_i)}, \frac{(1-T)Y(0)}{1-e(X_i)}])$$

2. Show that $$E[(\frac{TY(1)}{e(X_i)})^2] = E[\frac{(Y(1))^2}{e(X_i)}]$$

$$E[(\frac{TY(1)}{e(X_i)})^2] = E[\frac{T^2Y(1)^2}{e(X_i)^2}] = \frac{1}{e(X_i)^2} E[T^2 Y(1)^2] =\frac{1}{e(X_i)^2} E[T^2] E[Y(1)^2]  =\frac{1}{e(X_i)^2} E[T] E[Y(1)^2] =  \frac{1}{e(X_i)} E[Y(1)^2] = E[\frac{Y(1)^2}{e(X_i)}] $$

3. We admit similarly that $E[(\frac{(1-T)Y(0)}{1-e(X_i)})^2] = E[\frac{(Y(0))^2}{1-e(X_i)}]$. Using the general property of variance, show that $Var[\frac{TY(1)}{e(X_i)}] = E[\frac{(Y(1))^2}{e(X_i)}] - E[Y(1)]^2$ and that $Var[\frac{(1-T)Y(0)}{1-e(X_i)}] = E[\frac{(Y(0))^2}{1-e(X_i)}] - E[Y(0)]^2$

$$Var[\frac{TY(1)}{e(X_i)}] = E[\frac{T^2Y(1)^2}{e(X_i)^2}] - E[\frac{TY(1)}{e(X_i)}]^2 = E[\frac{(Y(1))^2}{e(X_i)}]  - \frac{1}{e(X_i)^2} E[TY(1)]^2 = E[\frac{(Y(1))^2}{e(X_i)}]  - \frac{1}{e(X_i)^2} E[T]^2 E[Y(1)]^2 = E[\frac{(Y(1))^2}{e(X_i)}] - E[Y(1)]^2$$

$$Var[\frac{(1-T)Y(0)}{1-e(X_i)}] = E[\frac{(1-T)^2Y(0)^2}{(1-e(X_i))^2}] - E[\frac{(1-T)Y(0)}{1-e(X_i)}]^2 = E[\frac{(Y(0))^2}{1-e(X_i)}] - E[\frac{(1-T)Y(0)}{1-e(X_i)}]^2 = E[\frac{(Y(0))^2}{1-e(X_i)}] - (\frac{1}{1-e(X_i)})^2E[(1-T)]^2E[Y(0)]^2$$
$$=E[\frac{Y(0)^2}{1-e(X_i)}] - E[Y(0)]^2$$

4. We admit that $Cov[ \frac{TY(1)}{e(X_i)}, \frac{(1-T)Y(1)}{1-e(X_i)}] = -E[Y(0)]E[Y(1)]$. Conclude the proof.


$$nVar[\hat{\tau}^*_{IPW}] = n\cdot \frac{1}{n}(Var[\frac{TY(1)}{e(X_i)}] + Var[\frac{(1-T)Y(0)}{1-e(X_i)}] - 2Cov[ \frac{TY(1)}{e(X_i)}, \frac{(1-T)Y(0)}{1-e(X_i)}])$$
$$= E[\frac{Y(1)^2}{e(X_i)}] - E[Y(1)]^2 +  E[\frac{(Y(0))^2}{1-e(X_i)}] - E[Y(0)]^2 +2E[Y(0)]E[Y(1)]$$
$$= E[\frac{Y(1)^2}{e(X_i)}] +  E[\frac{(Y(0))^2}{1-e(X_i)}] -  (E[Y(1)]^2 + E[Y(0)]^2 -2E[Y(0)]E[Y(1)]) $$
$$= E[\frac{Y(1)^2}{e(X_i)}] +  E[\frac{(Y(0))^2}{1-e(X_i)}] - (E[Y(1)-Y(0)])^2$$
$$= E[\frac{Y(1)^2}{e(X_i)}] +  E[\frac{(Y(0))^2}{1-e(X_i)}] - \tau^2$$

We could then use this result to compare the IPW estimator to other estimators for the ATE